![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [2]:
import pandas as pd
import numpy as np



In [3]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [4]:
client = df.copy()

In [5]:
client["job"] = client["job"].str.replace(".", "_")


In [6]:
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.nan)

In [7]:
client["credit_default"] = np.where(client["credit_default"] == "yes",1,0)

In [8]:
client["mortgage"] = np.where(client["mortgage"] == "yes",1,0)

In [9]:
client.to_csv("client.csv")

In [11]:
client = client.drop(columns=['campaign_outcome'])

In [12]:
client

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months
0,0,56,housemaid,married,basic_4y,0,0,may,13,261,1,0,nonexistent,93.994,4.857
1,1,57,services,married,high_school,0,0,may,19,149,1,0,nonexistent,93.994,4.857
2,2,37,services,married,high_school,0,1,may,23,226,1,0,nonexistent,93.994,4.857
3,3,40,admin_,married,basic_6y,0,0,may,27,151,1,0,nonexistent,93.994,4.857
4,4,56,services,married,high_school,0,0,may,3,307,1,0,nonexistent,93.994,4.857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,0,1,nov,30,334,1,0,nonexistent,94.767,1.028
41184,41184,46,blue-collar,married,professional_course,0,0,nov,6,383,1,0,nonexistent,94.767,1.028
41185,41185,56,retired,married,university_degree,0,1,nov,24,189,2,0,nonexistent,94.767,1.028
41186,41186,44,technician,married,professional_course,0,0,nov,17,442,1,0,nonexistent,94.767,1.028


![Campaign](abc.png)


In [13]:
campaign = df.copy()

In [14]:
campaign["previous_outcome"] = np.where(campaign["previous_outcome"] == "success",1,0)

In [15]:
campaign["campaign_outcome"] = np.where(campaign["campaign_outcome"] == "yes",1,0)

In [17]:
campaign["year"] = 2022

In [18]:
campaign["month"] = campaign["month"].map({
    'january': '01', 'february': '02', 'march': '03', 'april': '04',
    'may': '05', 'june': '06', 'july': '07', 'august': '08', 
    'september': '09', 'october': '10', 'november': '11', 'december': '12'
})

In [20]:
campaign["last_contact_date"] = campaign["year"].astype(str) + "-" + campaign["month"] + "-" + campaign["day"].astype(str)

In [21]:
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format='%Y-%m-%d')

In [22]:
campaign = campaign.drop(['age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'year','cons_price_idx','euribor_three_months'], axis=1)

In [ ]:
cam

In [24]:
campaign.to_csv("campaign.csv")

In [27]:
campaign

,client_id,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,261,1,0,0,0,2022-05-13
1,1,149,1,0,0,0,2022-05-19
2,2,226,1,0,0,0,2022-05-23
3,3,151,1,0,0,0,2022-05-27
4,4,307,1,0,0,0,2022-05-03
...,...,...,...,...,...,...,...
41183,41183,334,1,0,0,1,NaT
41184,41184,383,1,0,0,0,NaT
41185,41185,189,2,0,0,0,NaT
41186,41186,442,1,0,0,1,NaT


client_id	integer

cons_price_idx	float

euribor_three_months	float

In [25]:
economics = df[["client_id", "cons_price_idx", "euribor_three_months"]]

In [26]:
economics.to_csv("economics.csv")